# Spark

# Librerias

In [1]:
# general
from datetime import datetime, date
import pandas as pd

# etl
import pyspark 
from pyspark import SparkContext, SparkConf

from pyspark.sql import Row
from pyspark.sql import Column
from pyspark.sql.functions import upper
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import expr
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf



In [2]:
session = SparkSession.builder.master('local').appName('app1').getOrCreate()  
session

: 

In [5]:
# iniciar sesion de spark
spark = SparkSession.builder.getOrCreate()
#spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark

: 

In [2]:
install_requires=[
        'pyspark=={site.SPARK_VERSION}'
    ]
install_requires

['pyspark=={site.SPARK_VERSION}']

In [3]:
# crear PySpark DataFrame con lista de filas
data = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
data

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [4]:
# crear PySpark DataFrame con esquema explicito
data = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')
data


DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [5]:
# crear PySpark DataFrame de pandas DataFrame
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
data = spark.createDataFrame(pandas_df)
data

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [8]:
data.show

<bound method DataFrame.show of DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]>

In [9]:
data.printSchema

<bound method DataFrame.printSchema of DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]>

In [10]:
# convertir PySpark DataFrame a Pandas DataFrame
data.a

Column<'a'>

In [12]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [13]:
data

Py4JJavaError: An error occurred while calling o89.htmlString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChec

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [7]:
@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    return series + 1

In [9]:
# crear Spark DataFrame 
data = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
data

DataFrame[color: string, fruit: string, v1: bigint, v2: bigint]

In [16]:
# crear Dataframe de archivo de texto
data_text = spark.read.text("data/buddhist.txt")
data_text

DataFrame[value: string]

In [18]:
data_text.count() # contar el numero de filas
data_text.first() # ver primera fila

Row(value='Buddhist universities and colleges in the United States')

In [19]:
# filtrar datos con patron
linesWithSpark = data_text.filter(data_text.value.contains("Spark"))
linesWithSpark

DataFrame[value: string]

In [21]:
# ver fila con mas palabras
data_text.select(sf.size(sf.split(data_text.value,"\s+")).name("numWords")).agg(sf.max(sf.col("numWords"))).collect()

[Row(max(numWords)=41)]

In [22]:
linesWithSpark.cache()

DataFrame[value: string]

In [23]:
linesWithSpark.count() # contar filas con patron

0

In [24]:
# crear aplicacion de Spark simple 

logFile = "YOUR_SPARK_HOME/README.md"  # Should be some file on your system
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()
logData = spark.read.text(logFile).cache()

numAs = logData.filter(logData.value.contains('a')).count()
numBs = logData.filter(logData.value.contains('b')).count()

print("Lines with a: %i, lines with b: %i" % (numAs, numBs))

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/c:/Users/dfoso/Documents/GitHub/tech_skills/etl/YOUR_SPARK_HOME/README.md.